# {mod}`ultralytics` 追踪模式

追踪模式：将物体检测模式扩展到实时追踪应用中。

视频分析领域中的物体跟踪是一项关键任务，它不仅要识别帧内物体的位置和类别，还要在视频播放过程中为每个检测到的物体保留唯一的 ID。从监控和安防到实时体育分析，其应用是无限的。


```{admonition} 为什么选择Ultralytics YOLO进行目标跟踪？

Ultralytics追踪器的输出与标准[目标检测](https://www.ultralytics.com/glossary/object-detection)一致，但增加了对象ID的价值。这使得在视频流中跟踪对象并进行后续分析变得容易。以下是您应该考虑使用Ultralytics YOLO满足目标跟踪需求的原因：

- **效率：** 实时处理视频流，同时不牺牲[准确性](https://www.ultralytics.com/glossary/accuracy)。
- **灵活性：** 支持多种跟踪算法和配置。
- **易用性：** 简单的Python API和CLI选项，便于快速集成和部署。
- **可定制性：** 易于使用自定义训练的YOLO模型，允许集成到特定领域的应用中。
```


```{admonition} 功能特点

Ultralytics YOLO将其目标检测功能扩展，提供了强大且多功能的目标跟踪：

- **实时跟踪：**在高帧率视频中无缝跟踪对象。
- **多跟踪器支持：**从多种成熟的跟踪算法中选择。
- **可定制的跟踪器配置：**通过调整各种参数，定制跟踪算法以满足特定需求。
```

In [ ]:
import set_env

## 可用跟踪器

Ultralytics YOLO支持以下跟踪算法。可以通过传递相关的YAML配置文件来启用它们，例如使用`tracker=tracker_type.yaml`:

- [BoT-SORT](https://github.com/NirAharon/BoT-SORT) - 使用`botsort.yaml`文件来启用此跟踪器。
- [ByteTrack](https://github.com/ifzhang/ByteTrack) - 使用`bytetrack.yaml`文件来启用此跟踪器。

默认的跟踪器是BoT-SORT。

```{admonition} 目标跟踪阈值信息

如果对象置信度分数较低，即低于 [`track_high_thresh`](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/trackers/bytetrack.yaml#L5)，则不会有成功返回和更新的轨迹。
```

要在视频流上运行追踪器，可以使用经过训练的检测、分割或姿态模型，例如 YOLOv11n、YOLOv11n-seg 和 YOLOv11n-pose。

```python
from ultralytics import YOLO

# Load an official or custom model
model = YOLO("yolo11n.pt")  # Load an official Detect model
model = YOLO("yolo11n-seg.pt")  # Load an official Segment model
model = YOLO("yolo11n-pose.pt")  # Load an official Pose model
model = YOLO("path/to/best.pt")  # Load a custom trained model

# Perform tracking with the model
results = model.track("https://youtu.be/LNwODJXcvt4", show=True)  # Tracking with default tracker
results = model.track("https://youtu.be/LNwODJXcvt4", show=True, tracker="bytetrack.yaml")  # with ByteTrack
```

从上述使用情况可以看出，对于在视频或流媒体源上运行的所有检测、分割和姿态模型，均可进行追踪。

## 追踪任务

跟踪配置与预测模式共享属性，例如 `conf`, `iou` 和 `show`。

```python
# pip install lap
from ultralytics import YOLO

# Configure the tracking parameters and run the tracker
model = YOLO("yolo11n.pt")
results = model.track(source="https://youtu.be/LNwODJXcvt4", conf=0.3, iou=0.5, show=True)
```

### 追踪器选择

Ultralytics还允许您使用修改后的追踪器配置文件。为此，只需从[ultralytics/cfg/trackers](https://github.com/ultralytics/ultralytics/tree/main/ultralytics/cfg/trackers)复制一个追踪器配置文件（例如，`custom_tracker.yaml`），并根据需要修改任何配置（除了`tracker_type`）。

```python
from ultralytics import YOLO

# Load the model and run the tracker with a custom configuration file
model = YOLO("yolo11n.pt")
results = model.track(source="https://youtu.be/LNwODJXcvt4", tracker="custom_tracker.yaml")
```

关于追踪参数的完整列表，请参阅[ultralytics/cfg/trackers](https://github.com/ultralytics/ultralytics/tree/main/ultralytics/cfg/trackers)页面。

## Python 示例

### 持续追踪循环

以下是一个使用[OpenCV](https://www.ultralytics.com/glossary/opencv) (`cv2`) 和 YOLO11 在视频帧上运行对象追踪的 Python 脚本。此脚本假设您已经安装了必要的包（`opencv-python` 和 `ultralytics`）。参数 `persist=True` 告诉追踪器当前图像或帧是序列中的下一个，并期望在当前图像中看到来自前一图像的追踪轨迹。

```python
import cv2

from ultralytics import YOLO

# Load the YOLO11 model
model = YOLO("yolo11n.pt")

# Open the video file
video_path = "path/to/video.mp4"
cap = cv2.VideoCapture(video_path)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLO11 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()
```

请注意，`model(frame)`已更改为`model.track(frame)`，这一改动使得脚本能够进行目标追踪而不仅仅是简单的检测。修改后的脚本将在视频的每一帧上运行追踪器，可视化结果，并在窗口中展示。通过按下“q”键可以退出循环。

### 随时间绘制轨迹

通过连续帧可视化对象轨迹，可以深入了解视频中检测到的对象的运动模式和行为。使用Ultralytics YOLO11，绘制这些轨迹是一个无缝且高效的过程。

在以下示例中，我们展示了如何利用YOLO11的追踪功能，绘制检测对象在多个视频帧中的运动路径。该脚本涉及打开一个视频文件，逐帧读取，并使用YOLO模型识别和跟踪各种对象。通过保留检测到的边界框的中心点并将它们连接起来，我们可以绘制出代表被追踪对象路径的线条。

```python
from collections import defaultdict

import cv2
import numpy as np

from ultralytics import YOLO

# Load the YOLO11 model
model = YOLO("yolo11n.pt")

# Open the video file
video_path = "path/to/video.mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO11 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Display the annotated frame
        cv2.imshow("YOLO11 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()
```

### 多线程跟踪

多线程跟踪提供了在多个视频流上同时运行对象跟踪的能力。这在处理来自多个监控摄像头的视频输入时特别有用，其中并发处理可以显著提高效率和性能。

在提供的Python脚本中，我们使用Python的`threading`模块来同时运行多个跟踪器实例。每个线程负责在一个视频文件上运行跟踪器，并且所有线程都在后台同时运行。

为了确保每个线程接收到正确的参数（视频文件、使用的模型和文件索引），我们定义了一个函数`run_tracker_in_thread`，该函数接受这些参数并包含主要的跟踪循环。这个函数逐帧读取视频，运行跟踪器，并显示结果。

在此示例中使用了两个不同的模型：`yolo11n.pt`和`yolo11n-seg.pt`，它们分别在不同的视频文件中跟踪对象。视频文件在`video_file1`和`video_file2`中指定。

`threading.Thread`中的`daemon=True`参数意味着这些线程将在主程序结束时立即关闭。然后我们通过调用`start()`启动线程，并使用`join()`使主线程等待直到两个跟踪器线程都完成。

最后，在所有线程完成任务后，使用`cv2.destroyAllWindows()`关闭显示结果的窗口。

```python
import threading

import cv2

from ultralytics import YOLO

# Define model names and video sources
MODEL_NAMES = ["yolo11n.pt", "yolo11n-seg.pt"]
SOURCES = ["path/to/video.mp4", "0"]  # local video, 0 for webcam


def run_tracker_in_thread(model_name, filename):
    """
    Run YOLO tracker in its own thread for concurrent processing.

    Args:
        model_name (str): The YOLO11 model object.
        filename (str): The path to the video file or the identifier for the webcam/external camera source.
    """
    model = YOLO(model_name)
    results = model.track(filename, save=True, stream=True)
    for r in results:
        pass


# Create and start tracker threads using a for loop
tracker_threads = []
for video_file, model_name in zip(SOURCES, MODEL_NAMES):
    thread = threading.Thread(target=run_tracker_in_thread, args=(model_name, video_file), daemon=True)
    tracker_threads.append(thread)
    thread.start()

# Wait for all tracker threads to finish
for thread in tracker_threads:
    thread.join()

# Clean up and close windows
cv2.destroyAllWindows()
```

这个例子可以很容易地通过创建更多的线程并应用相同的方法来扩展，以处理更多的视频文件和模型。

## 贡献新的追踪器

你是否精通多目标追踪，并且已经成功实现或适配了Ultralytics YOLO的追踪算法？我们邀请你为[ultralytics/cfg/trackers](https://github.com/ultralytics/ultralytics/tree/main/ultralytics/cfg/trackers)中的追踪器部分做出贡献！你的实际应用和解决方案对于从事追踪任务的用户来说可能非常有价值。


让我们一起增强Ultralytics YOLO生态系统的追踪能力&#128591;！

[鱼追踪]: https://github.com/RizwanMunawar/ultralytics/assets/62513924/a5146d0f-bfa8-4e0a-b7df-3c1446cd8142
[人追踪]: https://github.com/RizwanMunawar/ultralytics/assets/62513924/93bb4ee2-77a0-4e4e-8eb6-eb8f527f0527
[车辆追踪]: https://github.com/RizwanMunawar/ultralytics/assets/62513924/ee6e6038-383b-4f21-ac29-b2a1c7d386ab

## 常见问题解答

### 什么是多目标追踪及其如何被Ultralytics YOLO支持？

视频分析中的多目标追踪涉及识别对象并在视频帧中为每个检测到的对象保持唯一ID。Ultralytics YOLO通过提供实时追踪以及对象ID来支持这一功能，便于执行如安全监控和体育分析等任务。系统使用如BoT-SORT和ByteTrack之类的追踪器，这些可以通过YAML文件进行配置。

### 如何配置Ultralytics YOLO的自定义跟踪器？

您可以通过复制现有的跟踪器配置文件（例如，`custom_tracker.yaml`）并从[Ultralytics跟踪器配置目录](https://github.com/ultralytics/ultralytics/tree/main/ultralytics/cfg/trackers)进行必要的参数修改（除了`tracker_type`之外），来配置自定义跟踪器。在您的跟踪模型中使用此文件，如下所示：

```python
from ultralytics import YOLO

model = YOLO("yolo11n.pt")
results = model.track(source="https://youtu.be/LNwODJXcvt4", tracker="custom_tracker.yaml")
```

### 如何在多个视频流上同时运行对象跟踪？

要同时在多个视频流上运行对象跟踪，您可以使用Python的`threading`模块。每个线程将处理一个单独的视频流。以下是设置的示例：

```python
import threading

import cv2

from ultralytics import YOLO

# 定义模型名称和视频源
MODEL_NAMES = ["yolo11n.pt", "yolo11n-seg.pt"]
SOURCES = ["path/to/video.mp4", "0"]  # 本地视频，0为摄像头

def run_tracker_in_thread(model_name, filename):
    """
    在自己的线程中运行YOLO跟踪器以实现并发处理。

    参数:
        model_name (str): YOLO11模型对象。
        filename (str): 视频文件的路径或摄像头/外部摄像头源的标识符。
    """
    model = YOLO(model_name)
    results = model.track(filename, save=True, stream=True)
    for r in results:
        pass

# 创建并启动跟踪器线程
tracker_threads = []
for video_file, model_name in zip(SOURCES, MODEL_NAMES):
    thread = threading.Thread(target=run_tracker_in_thread, args=(model_name, video_file), daemon=True)
    tracker_threads.append(thread)
    thread.start()

# 等待所有跟踪器线程完成
for thread in tracker_threads:
    thread.join()

# 清理并关闭窗口
cv2.destroyAllWindows()
```

### **Ultralytics YOLO多目标追踪在现实世界中的应用有哪些？**

Ultralytics YOLO的多目标追踪技术在多个领域都有实际应用，包括：

- **交通：** 车辆跟踪用于交通管理和自动驾驶。
- **零售：** 人员跟踪用于店内分析和安全监控。
- **水产养殖：** 鱼类跟踪用于监测水生环境。

这些应用得益于Ultralytics YOLO能够实时处理高帧率视频的能力。

### **如何使用Ultralytics YOLO在多个视频帧中可视化对象轨迹？**

要使用Ultralytics YOLO在多个视频帧中可视化对象轨迹，您可以利用YOLO模型的追踪功能以及OpenCV绘制检测到的对象路径。

```python
from collections import defaultdict

import cv2
import numpy as np

from ultralytics import YOLO

model = YOLO("yolo11n.pt")
video_path = "path/to/video.mp4"
cap = cv2.VideoCapture(video_path)
track_history = defaultdict(lambda: [])

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True)
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()
        annotated_frame = results[0].plot()
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))
            if len(track) > 30:
                track.pop(0)
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
        cv2.imshow("YOLO11 Tracking", annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()
```

这个脚本将绘制跟踪线，展示随时间推移被追踪对象的移动路径。